## Training the parking lot occupancy recognition system using `Google Colab`, **Ultralytics** (YOLOv8) and **ClearML** libraries

In [ ]:
%pip install ultralytics clearml

In [ ]:
# Read the docs for details https://clear.ml/docs/latest/docs/getting_started/ds/ds_first_steps/
%clearml-init

import clearml
clearml.browser_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%mkdir -p /content/datasets/parking_dataset

import torch

# Download a training dataset (1921 images)
torch.hub.download_url_to_file(
    'https://universe.roboflow.com/ds/jZph9wFmK8?key=Pf6FiiVhjo',
    '/content/datasets/parking_dataset.zip'
)

# Unzip and remove an archive
%unzip \
  -q /content/datasets/parking_dataset.zip \
  -d /content/datasets/parking_dataset \
  && rm /content/datasets/parking_dataset.zip

# Set a correct path to the dataset
%sed \
  -i '1,4c\path: /content/datasets/parking_dataset/\ntrain: train/images\nval: valid/images\ntest: test/images\n' \
  /content/datasets/parking_dataset/data.yaml

In [ ]:
from clearml import Task
from ultralytics import YOLO

# Set the working directory where the training results will be stored
working_dir = ""

model = YOLO("yolov8m.pt")  # Load a pretrained model (recommended for training)
task = Task.init(project_name="SpotGazer", task_name="Parking occupancy detection training")

model.train(  # Train the model
    data="/content/datasets/parking_dataset/data.yaml",
    epochs=75,
    imgsz=640,
    project=f"{working_dir}/yolov8m_spot_gazer",
)